In [7]:
# Update package list and install necessary dependencies
!apt update && apt install -y \
    git \
    ffmpeg \
    nodejs \
    npm \
    wget \
    curl \
    unzip \
    chromium-chromedriver  # Install Chromium and ChromeDriver

# Install Node.js packages
!npm install -g puppeteer

# Install Python packages
!pip install gitpython selenium  # Install GitPython and Selenium packages


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
35 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [25]:
import subprocess
import time
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from git import Repo

# Cloning the repository from GitHub
REPO_URL = 'https://github.com/JerusalemHills/jerusalemhills.github.io.git'
REPO_DIR = '/content/jerusalemhills.github.io'  # Store the repo in Colab environment
OUTPUT_VIDEO = "screenshots/timelapse.mp4"
SCREENSHOT_DIR = os.path.join(REPO_DIR, "screenshots")

# Clone the repository (this will fetch the latest version)
if not os.path.exists(REPO_DIR):
    print(f"Cloning repository from {REPO_URL}...")
    subprocess.run(["git", "clone", REPO_URL, REPO_DIR])

# Ensure the screenshots directory exists
if not os.path.exists(SCREENSHOT_DIR):
    os.makedirs(SCREENSHOT_DIR)

# Function to start simple HTTP server for `index.html`
def start_http_server():
    """Start an HTTP server to serve index.html and related assets."""
    os.chdir(REPO_DIR)
    print("Starting HTTP server...")
    server_process = subprocess.Popen(["python3", "-m", "http.server", "8000"])
    return server_process

# Function to capture screenshot using headless Chrome (Puppeteer)
def capture_screenshot(commit_hash):
    """Capture screenshot of index.html for the given commit hash."""
    screenshot_path = os.path.join(SCREENSHOT_DIR, f"screenshot_{commit_hash}.png")

    # Configure headless Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # Create webdriver
    driver = webdriver.Chrome(options=chrome_options)

    try:
        # Load index.html served by the HTTP server
        driver.get("http://localhost:8000/index.html")
        time.sleep(3)  # Wait for page to fully render
        driver.save_screenshot(screenshot_path)  # Save screenshot
        print(f"Screenshot saved for commit {commit_hash} at {screenshot_path}")
    except Exception as e:
        print(f"Error taking screenshot for commit {commit_hash}: {e}")
    finally:
        driver.quit()

# Function to upload screenshots and video to the repository
def upload_to_repo():
    """Commits and pushes generated screenshots and video to the repository."""
    repo = Repo(REPO_DIR)
    repo.git.add(SCREENSHOT_DIR)
    repo.git.add(OUTPUT_VIDEO)
    repo.git.commit("-m", "Added timelapse video and screenshots")
    repo.git.push()
    print("✅ Screenshots and video uploaded to repository.")

# Main function to drive the process
def main():
    """Main execution function."""
    repo = Repo(REPO_DIR)
    commits = list(repo.iter_commits('master'))[::-1]  # From oldest to newest

    # Start HTTP server to serve the repo's files
    server_process = start_http_server()
    time.sleep(3)  # Wait for server to start

    for commit in commits:
        repo.git.checkout(commit.hexsha)

        # Capture screenshot of index.html for this commit
        capture_screenshot(commit.hexsha)

    # Terminate the server
    server_process.terminate()

    # Generate video (implement the `generate_video()` function as needed)
    generate_video()

    print(f"✅ Timelapse video saved as {OUTPUT_VIDEO}")

    # Upload screenshots and video to the repo
    upload_to_repo()

if __name__ == "__main__":
    main()


OSError: Lock for file '/content/jerusalemhills.github.io/.git/config' did already exist, delete '/content/jerusalemhills.github.io/.git/config.lock' in case the lock is illegal

In [21]:
!find /content/ -type d -name "*screenshot*"


/content/screenshots
/content/jerusalemhills.github.io/screenshots


In [22]:
!ls /content/jerusalemhills.github.io/screenshots

screenshot_009ebd81aa60c7749017a487742505b0f49ad749.png
screenshot_01554f0e7b1a314f8253fe4be1207dd460e2a9cb.png
screenshot_01aba5e3d08566204536dca38d65fc0f458d9dcb.png
screenshot_01e91fefb65d25d2bd0923dee61dc4ab445a8710.png
screenshot_07519769f43b1e033b62ed731a30ccc1a37a10f5.png
screenshot_086be752a4d41e265283b23fafc37ac63a723ba4.png
screenshot_08f0b9b8790d75992784ad49a9632737bef9502d.png
screenshot_08f5f0de172d321ecf6f58c3641ad26a9a0c4016.png
screenshot_0d33b8b86c41bb613f9c65b1e25da32fcce956dc.png
screenshot_0e44c162eefe236bc3ba058bdb043daa9a50cdf0.png
screenshot_0ecd5d269c0f29d3cfa6ccf0e4be102ef4f7147c.png
screenshot_11290bd2d3daf920903cd27a25237ac0ecb48db6.png
screenshot_129b074a01186efe587865777811c679910f4876.png
screenshot_14a11c89883f794408c4046981f1f1633e9af033.png
screenshot_161aeb3ebe34e4c457de529883c4ddd38d5d80d9.png
screenshot_1650ff6de22a5e0d6af87e3b695a3f24ce05d5a4.png
screenshot_186afe1819fa22b21c6496b2886b0b4407555657.png
screenshot_18a23c992a46f49d63c29b6820e3366c14ec9

In [24]:
import shutil
import os
from google.colab import files

# Define the folder that contains the screenshots and video
source_directory = "/content/jerusalemhills.github.io/screenshots"
zip_file_path = "/content/screenshots.zip"

# Check if the folder exists and contains files
if os.path.exists(source_directory) and os.listdir(source_directory):
    # Create a zip file of the folder (ensure the folder path is correct)
    shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', source_directory)

    # Initiate the download
    files.download(zip_file_path)
else:
    print(f"Folder {source_directory} does not exist or is empty.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>